In [232]:
import os

from dotenv import load_dotenv
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits.pandas.base import _get_functions_prompt_and_tools
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.agents import AgentExecutor, AgentType
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory



import pandas as pd


load_dotenv('../.flaskenv')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [240]:
embeddings = OpenAIEmbeddings()
persist_directory = '/home/fasih/ADIB/chroma4'

adib = pd.read_csv('/home/fasih/ADIB/data/ADIB_Financial data_With cost heads_V2.csv')

In [241]:
adib.iloc[:,1:-2].astype(float).head()

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,4351.609,4620.741,4631.464,3826.839,3674.567,5041.871,5646.89552,6324.522982,7083.465740,7933.481629,8885.499425
1,581.982,714.034,813.211,502.358,329.959,890.951,997.86512,1117.608934,1251.722007,1401.928647,1570.160085
2,3769.627,3906.707,3818.253,3324.481,3344.608,4150.920,4649.03040,5206.914048,5831.743734,6531.552982,7315.339340
3,545.979,525.465,686.971,791.385,994.310,1471.339,1647.89968,1845.647642,2067.125359,2315.180402,2593.002050
4,1030.268,1058.665,1083.270,992.163,958.210,1368.348,1532.54976,1716.455731,1922.430419,2153.122069,2411.496718


In [242]:
adib.index = adib['Chart of Accounts']
adib.head()

,Chart of Accounts,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,YOY 2022
Chart of Accounts,,,,,,,,,,,,,,
Gross revenue from funds,Gross revenue from funds,4351.609,4620.741,4631.464,3826.839,3674.567,5041.871,5646.89552,6324.522982,7083.465740,7933.481629,8885.499425,9951.759356,37%
Distribution to depositors,Distribution to depositors,581.982,714.034,813.211,502.358,329.959,890.951,997.86512,1117.608934,1251.722007,1401.928647,1570.160085,1758.579295,170%
Funded income,Funded income,3769.627,3906.707,3818.253,3324.481,3344.608,4150.920,4649.03040,5206.914048,5831.743734,6531.552982,7315.339340,8193.180060,24%
Investment income,Investment income,545.979,525.465,686.971,791.385,994.310,1471.339,1647.89968,1845.647642,2067.125359,2315.180402,2593.002050,2904.162296,48%
Fees and commission income,Fees and commission income,1030.268,1058.665,1083.270,992.163,958.210,1368.348,1532.54976,1716.455731,1922.430419,2153.122069,2411.496718,2700.876324,43%


In [243]:
adib = adib.iloc[:,:14]
adib.head()

,Chart of Accounts,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,YOY 2022
Chart of Accounts,,,,,,,,,,,,,,
Gross revenue from funds,Gross revenue from funds,4351.609,4620.741,4631.464,3826.839,3674.567,5041.871,5646.89552,6324.522982,7083.465740,7933.481629,8885.499425,9951.759356,37%
Distribution to depositors,Distribution to depositors,581.982,714.034,813.211,502.358,329.959,890.951,997.86512,1117.608934,1251.722007,1401.928647,1570.160085,1758.579295,170%
Funded income,Funded income,3769.627,3906.707,3818.253,3324.481,3344.608,4150.920,4649.03040,5206.914048,5831.743734,6531.552982,7315.339340,8193.180060,24%
Investment income,Investment income,545.979,525.465,686.971,791.385,994.310,1471.339,1647.89968,1845.647642,2067.125359,2315.180402,2593.002050,2904.162296,48%
Fees and commission income,Fees and commission income,1030.268,1058.665,1083.270,992.163,958.210,1368.348,1532.54976,1716.455731,1922.430419,2153.122069,2411.496718,2700.876324,43%


In [268]:
cost_columns = adib['Chart of Accounts'].str.contains('(Cost)', regex=True)
operating_cost_columns = ['Card related fees and commission expense (Cost)', 'Other fees and commission expenses (Cost)', 'Cost-to-Income Ratio (Cost)', 'Efficiency Ratio (Cost)', 'Operating Profit Margin (Cost)']
operating_cost_2020 = adib.loc[cost_columns & ~adib['Chart of Accounts'].isin(operating_cost_columns), '2020'].sum()
operating_cost_2023 = adib.loc[cost_columns & ~adib['Chart of Accounts'].isin(operating_cost_columns), '2023'].sum()
percentage_increase = ((operating_cost_2023 - operating_cost_2020) / operating_cost_2020) * 100
percentage_increase

/tmp/ipykernel_1511679/2692311490.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = adib['Chart of Accounts'].str.contains('(Cost)', regex=True)


-6.106802348070409

Chart of Accounts
Gross revenue from funds                                           False
Distribution to depositors                                         False
Funded income                                                      False
Investment income                                                  False
Fees and commission income                                         False
Foreign exchange income                                            False
Other income                                                       False
Non-funded income                                                  False
Revenues                                                           False
Net income before zakat and tax                                    False
Zakat and tax                                                      False
Net income after zakat and tax                                     False
Non-controlling interest                                           False
Net income attributable to equity

In [273]:
employee_costs_2020 = adib.loc[adib['Chart of Accounts'].str.contains('Salaries and wages (Cost)', regex=False) | adib['Chart of Accounts'].str.contains('End of service benefits  (Cost)', regex=False) | adib['Chart of Accounts'].str.contains('Other staff expenses (Cost)', regex=False), '2020'].sum()
employee_costs_2023 = adib.loc[adib['Chart of Accounts'].str.contains('Salaries and wages (Cost)', regex=False) | adib['Chart of Accounts'].str.contains('End of service benefits  (Cost)', regex=False) | adib['Chart of Accounts'].str.contains('Other staff expenses (Cost)', regex=False), '2023'].sum()
percentage_increase = ((employee_costs_2023 - employee_costs_2020) / employee_costs_2020) * 100
percentage_increase

14.427117673112264

In [220]:
adib[adib['Chart of Accounts'].str.endswith('(Cost)')].nlargest(3, '2021')

,Chart of Accounts,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028
Chart of Accounts,,,,,,,,,,,,,
Operating expenses (Cost),Operating expenses (Cost),2509.168,2643.781,2653.051,2450.000,2260.035,2386.720,2673.12640,2993.901568,3353.169756,3755.550127,4206.216142,4710.962079
Employees costs (Cost),Employees costs (Cost),1446.329,1522.644,1529.670,1508.212,1438.767,1540.896,1725.80352,1932.899942,2164.847935,2424.629688,2715.585250,3041.455480
Salaries and wages (Cost),Salaries and wages (Cost),1310.133,1380.252,1391.292,1365.404,1321.467,1415.328,1585.16736,1775.387443,1988.433936,2227.046009,2494.291530,2793.606513


In [213]:
cost_columns

[]

Chart of Accounts
Gross revenue from funds                                           False
Distribution to depositors                                         False
Funded income                                                      False
Investment income                                                  False
Fees and commission income                                         False
Foreign exchange income                                            False
Other income                                                       False
Non-funded income                                                  False
Revenues                                                           False
Net income before zakat and tax                                    False
Zakat and tax                                                      False
Net income after zakat and tax                                     False
Non-controlling interest                                           False
Net income attributable to equity

In [262]:
]].sum()

/tmp/ipykernel_1511679/3635972053.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  adib[adib['Chart of Accounts'].str.contains('(Cost)',regex=True)]['2021'].sum()- adib[['Card related fees and commission expense (Cost)','Other fees and commission expenses (Cost)','Cost-to-Income Ratio (Cost)','Efficiency Ratio (Cost)', 'Operating Profit Margin (Cost)']].sum()


KeyError: "None of [Index(['Card related fees and commission expense (Cost)',\n       'Other fees and commission expenses (Cost)',\n       'Cost-to-Income Ratio (Cost)', 'Efficiency Ratio (Cost)',\n       'Operating Profit Margin (Cost)'],\n      dtype='object')] are in the [columns]"

In [199]:
adib.columns

Index(['Chart of Accounts', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', '2024', '2025', '2026', '2027', '2028'],
      dtype='object')

In [200]:
adib.dtypes

Chart of Accounts     object
2017                 float64
2018                 float64
2019                 float64
2020                 float64
2021                 float64
2022                 float64
2023                 float64
2024                 float64
2025                 float64
2026                 float64
2027                 float64
2028                 float64
dtype: object

In [201]:
prefix = ("You are the CFO of the bank and you're analyzing the balance sheet data."
          "Here are some insights from the data:"
          "- You can use the Pandas DataFrame named 'adib' to analyze financial data."
          "- Use appropriate filters to focus on specific information."
          "- Columns are:'Chart of Accounts','2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025','2026', '2027', '2028'"
          """- 'Chart of Accounts' contain these values: 
'Gross revenue from funds', 'Distribution to depositors',
'Funded income', 'Investment income', 'Fees and commission income',
'Foreign exchange income', 'Other income', 'Non-funded income',
'Revenues', 'Net income before zakat and tax', 'Zakat and tax',
'Net income after zakat and tax', 'Non-controlling interest',
'Net income attributable to equity holders of the bank',
'Cash and balances with central banks',
'Due from financial institutions', 'Customer financing', 'Investments',
'Investment in associates', 'Investment and development properties',
'Other assets/ Property and Equipement/ Goodwill and Intangibles',
'Total assets', 'Due to financial institutions', 'Total deposits',
'Other liabilities', 'Total liabilities', 'Share capital ',
'Legal reserve', 'General reserve', 'Credit risk reserve',
'Retained earnings', 'Proposed dividends', 'Other reserves',
'Equity attributable to shareholders of the bank', 'Tier 1 sukuk ',
'Non - controlling interest',
'Equity attributable to equity holders of the bank',
'Total liabilities and equity', 'Customer financing gross',
'Non-performing financing (NPA)', 'NPA ratio', 'NPA coverage ratio',
'NPA coverage ratio with collaterals', 'RWA', 'Operating Income',
'Total Revenue', 'Tier 1 Capital Ratio', 'Capital Adequacy Ratio',
'Common Equity Tier 1 Ratio ', 'Depreciation    (Cost)',
'Amortisation of intangibles  (Cost)',
'Card related fees and commission expense (Cost)',
'Other fees and commission expenses (Cost)',
'Salaries and wages (Cost)', 'End of service benefits  (Cost)',
'Other staff expenses (Cost)', 'Legal and professional expenses (Cost)',
'Premises expenses (Cost)', 'Marketing and advertising expenses (Cost)',
'Communication expenses  (Cost)', 'Technology related expenses  (Cost)',
'Finance cost on lease liabilities  (Cost)',
'Other operating expenses  (Cost)',
'Murabaha and other Islamic financing (Cost)', 'Ijara financing (Cost)',
'Direct write-off net of recoveries (Cost)',
'Investment in sukuk measured at amortised cost (Cost)',
'Other Impairment charges (Cost)', 'Cost-to-Income Ratio (Cost)',
'Efficiency Ratio (Cost)', 'Operating Profit Margin (Cost)'"""
    "- current year is 2023 and the number against (Cost) chart of accounts are in AED Millions"
    "- for all cost related queries check against all index ending with '(Cost)'"
    "- if asked top cost heads/units/silos return the descending by numbers"
    "Following are the formulae for special cost heads:"
    "Operating expenses = Provision for impairment net + General and administrative expenses +Employees costs +Depreciation    (Cost)+Amortisation of intangibles  (Cost)"
    "Provision for impairment net = Murabaha and other Islamic financing (Cost)+ Ijara financing (Cost) +Direct write-off, net of recoveries (Cost)+Investment in sukuk measured at amortised cost (Cost)+Other Impairment charges (Cost)"
    "General and administrative expenses = Legal and professional expenses (Cost)+ Premises expenses (Cost) + Marketing and advertising expenses (Cost)+ Communication expenses  (Cost)+ Technology related expenses  (Cost)+Finance cost on lease liabilities  (Cost)+Other operating expenses"
    "Employees costs = Salaries and wages (Cost)+ End of service benefits  (Cost)+ Other staff expenses (Cost)"
)

system_message = SystemMessage(content=prefix)

MEMORY_KEY = "chat_history" 
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [202]:
df_tool = PythonAstREPLTool(
    locals={"adib":adib}, 
    description=("A Python shell. Use this to execute python commands."
#                  "Input should be a valid python command. When using this tool, sometimes output is abbreviated"
#                  "- make sure it does not look abbreviated before using it in your answer."
                )
)


In [203]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [204]:
agent = OpenAIFunctionsAgent(
            llm=llm,
            prompt=prompt,
            tools=[df_tool]
)

In [205]:
memory = ConversationBufferWindowMemory(memory_key=MEMORY_KEY, return_messages=True, k=4)

In [206]:
agent = AgentExecutor(
    agent=agent,
    tools=[df_tool],
    memory=memory,
    max_iterations=7,
    verbose=True
)

In [209]:
print(agent.run('what are my top 3 costs for current year by values?'))



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `adib[adib['Chart of Accounts'].str.endswith('(Cost)')].nlargest(3, '2023')`


            Chart of Accounts      2017      2018      2019      2020  \
49  Operating expenses (Cost)  2509.168  2643.781  2653.051  2450.000   
52   Employees costs   (Cost)  1446.329  1522.644  1529.670  1508.212   
57  Salaries and wages (Cost)  1310.133  1380.252  1391.292  1365.404   

        2021      2022        2023         2024         2025         2026  \
49  2260.035  2386.720  2673.12640  2993.901568  3353.169756  3755.550127   
52  1438.767  1540.896  1725.80352  1932.899942  2164.847935  2424.629688   
57  1321.467  1415.328  1585.16736  1775.387443  1988.433936  2227.046009   

           2027         2028  
49  4206.216142  4710.962079  
52  2715.585250  3041.455480  
57  2494.291530  2793.606513  The top 3 costs for the current year (2023) by values are:

1. Operating expenses (Cost): AED 2,673.13 million
2. Employee

KeyError: '2023'

In [12]:
agent.run("Which operating expenses have unexpectedly increased in the year 2022 compared to the previous three years?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `adib[['Periods', 'Operating expenses']].sort_values('Periods', ascending=False).head(4)`


    Periods  Operating expenses
11     2028             2006.62
10     2027             2062.76
9      2026             2118.90
8      2025             2175.05
Invoking: `search_adib_annual_report` with `operating expenses`


[Document(page_content='and resale of properties and all associated activities.\nOther operations - Other operations comprises mainly of Head Office, subsidiaries, associates and joint ventures other than above categories \nincluding unallocated costs.\nManagement monitors the operating results of the operating segments separately for the purpose of making decisions about resource allocation \nand performance assessment. Segment performance is evaluated based on operating profit or loss.\nBusiness segments information for the year ended 31 December 2018 were as follows:\nGlobal Retail\nbanking\nAED ‘0

'The operating expenses that have unexpectedly increased in the year 2022 compared to the previous three years are as follows:\n\n1. Global Retail banking: The operating expenses increased from AED 1,205,586 in 2021 to AED 1,809,154 in 2022.\n\n2. Global Wholesale banking: The operating expenses increased from AED 17,402 in 2021 to AED 275,499 in 2022.\n\n3. Private banking: The operating expenses increased from AED 81,167 in 2021 to AED 69,725 in 2022.\n\n4. Treasury: The operating expenses increased from AED 740,335 in 2021 to AED 598,830 in 2022.\n\n5. Real estate: The operating expenses increased from AED 3,428 in 2021 to AED 6,020 in 2022.\n\n6. Other operations: The operating expenses increased from AED 297,199 in 2021 to AED 594,051 in 2022.\n\nThese segments have experienced unexpected increases in operating expenses in the year 2022 compared to the previous three years.'

In [173]:
agent.run('What could be the potential strategies to minimize the costs incurred?')



> Entering new AgentExecutor chain...
To minimize costs incurred, here are some potential strategies that can be considered:

1. Review and optimize operational processes: Identify areas where processes can be streamlined or automated to reduce manual effort and associated costs. Look for opportunities to eliminate unnecessary steps, improve efficiency, and reduce waste.

2. Negotiate better supplier contracts: Review existing supplier contracts and negotiate better terms, including discounts, bulk purchasing options, or longer payment terms. Consider exploring alternative suppliers to ensure competitive pricing.

3. Implement cost control measures: Implement strict cost control measures across the organization. This can include setting spending limits, requiring approval for non-essential expenses, and regularly monitoring and analyzing expenses to identify areas of overspending.

4. Encourage cost-conscious culture: Foster a culture of cost-consciousness among employees. Educate an

"To minimize costs incurred, here are some potential strategies that can be considered:\n\n1. Review and optimize operational processes: Identify areas where processes can be streamlined or automated to reduce manual effort and associated costs. Look for opportunities to eliminate unnecessary steps, improve efficiency, and reduce waste.\n\n2. Negotiate better supplier contracts: Review existing supplier contracts and negotiate better terms, including discounts, bulk purchasing options, or longer payment terms. Consider exploring alternative suppliers to ensure competitive pricing.\n\n3. Implement cost control measures: Implement strict cost control measures across the organization. This can include setting spending limits, requiring approval for non-essential expenses, and regularly monitoring and analyzing expenses to identify areas of overspending.\n\n4. Encourage cost-conscious culture: Foster a culture of cost-consciousness among employees. Educate and train employees on the import

In [13]:
result = agent.run('Which components of the income statement, balance sheet, and cash flows have exhibited irregular trends over the past three years?')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'search_term': 'income statement'}`


[Document(page_content='Financial Statements Abridged Income StatementFor the year ended  \n31 December\n2020\nAED Million2021\nAED Million\nFees and commissions 45.1 45.4\nOther income  -    0.3 \nTotal Revenues 45.1 45.7 \nTotal expenses (48.9) (44.0)\n(Loss) / Profit for the year (3.8) 1.8 \n NON-FINANCIAL SERVICES \nBUSINESS REVIEW  \nCONTINUED\nADIB Annual Report 2021\n38', metadata={'page': 37, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='Financial StatementsAbridged Income StatementFor the year ended  \n31 December\n2021\nAED Million2022\nAED Million\nFees and commissions 45.4 46.4\nOther income  0.3  - \nTotal Revenues 45.7 46.4 \nTotal expenses (44.0) (41.1)\n(Loss) / Profit for the year 1.8 5.3 \n NON-FINANCIAL SERVICES \nBUSINESS REVIEW \nCONTINUED\nOPERATIONAL \nREVIEW \nRetail banking group \nWholesa

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4864 tokens (4725 in the messages, 139 in the functions). Please reduce the length of the messages or functions.

In [175]:
result = agent.run('What are the possible ways to minimize operating expenses for each operating segment')
result



> Entering new AgentExecutor chain...
To minimize operating expenses for each operating segment, here are some possible ways:

1. Customer Financing Segment:
   - Streamline loan origination and approval processes to reduce administrative costs.
   - Implement credit risk assessment tools and models to improve loan quality and reduce provisions for impairment.
   - Offer digital self-service options for customers to reduce the need for manual assistance and lower operational costs.
   - Optimize collection processes to minimize delinquencies and associated costs.

2. Investments Segment:
   - Utilize technology platforms and automated tools for investment research and analysis to reduce manual effort and associated costs.
   - Optimize portfolio management strategies to minimize transaction costs and improve investment returns.
   - Leverage data analytics to identify investment opportunities with higher potential returns and lower associated costs.
   - Regularly review and rebalanc

"To minimize operating expenses for each operating segment, here are some possible ways:\n\n1. Customer Financing Segment:\n   - Streamline loan origination and approval processes to reduce administrative costs.\n   - Implement credit risk assessment tools and models to improve loan quality and reduce provisions for impairment.\n   - Offer digital self-service options for customers to reduce the need for manual assistance and lower operational costs.\n   - Optimize collection processes to minimize delinquencies and associated costs.\n\n2. Investments Segment:\n   - Utilize technology platforms and automated tools for investment research and analysis to reduce manual effort and associated costs.\n   - Optimize portfolio management strategies to minimize transaction costs and improve investment returns.\n   - Leverage data analytics to identify investment opportunities with higher potential returns and lower associated costs.\n   - Regularly review and rebalance investment portfolios to 

In [178]:
result = agent.run('what cost heads do I have?')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `cost heads`


[Document(page_content='Sensitivity to a one percentage point changes in the discount rate or the terminal growth rate and based on the results; management \nbelieves that no reasonably possible change in any of the above mentioned key assumptions would cause the carrying value to exceed the \nrecoverable amount.\n27 DUE TO FINANCIAL INSTITUTIONS\n2018\nAED‘0002017\nAED‘000\nCurrent accounts 878,693 1,538,954\nInvestment deposits     3,203,909     2,149,604\n4,082,602 3,688,558\nCurrent account – Central Bank of UAE          55,652                    -\n    4,138,254     3,688,558', metadata={'page': 105, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2018.pdf'}), Document(page_content='been discounted at a rate of 4.5% (2019: 5.7%).\nSensitivity to a one percentage point changes in the discount rate or the terminal growth rate and based on the results; management \nbelieves

"Based on the available search results from the ADIB annual reports, the specific cost heads mentioned are as follows:\n\n1. Customer Financing Segment:\n   - Provision for impairment\n\n2. Investments Segment:\n   - Not mentioned in the available search results\n\n3. Retail Banking Segment:\n   - Not mentioned in the available search results\n\n4. Corporate Banking Segment:\n   - Not mentioned in the available search results\n\n5. Asset Management Segment:\n   - Not mentioned in the available search results\n\n6. Other Operating Segments:\n   - Property and equipment\n   - Depreciation\n\nPlease note that the search results may not cover all the cost heads mentioned in the ADIB annual reports. It's recommended to refer to the complete annual reports for a comprehensive understanding of the cost heads associated with each operating segment."

In [16]:
result = agent.run('what is the commission income in Abridged Income Statement under ADIB securities')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'text': 'commission income', 'section': 'Abridged Income Statement', 'subsection': 'ADIB securities'}`




ToolException: Too many arguments to single-input tool search_adib_annual_report. Args: ['commission income', 'Abridged Income Statement', 'ADIB securities']

In [17]:
result = agent.run('Tell me about Burooj Properties LLC.')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'query': 'Burooj Properties LLC'}`


[Document(page_content='Investment properties 22 - 625\nDevelopment properties 23 - 31,148\nOthers     15,753   100,758\n954,399 1,314,112\nThe above provision for impairment includes nil (2020: AED 31,773 thousand) pertaining to Burooj Properties LLC, a real estate subsidiary of \nthe Bank.', metadata={'page': 105, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='Burooj Properties LLC ** UAE 100 Real Estate Investments Not consolidated Fully consolidated\nMPM Properties LLC ** UAE 100 Real Estate Services Not consolidated Fully consolidated\nKawader Services LLC ** UAE 100 Manpower Supply Not consolidated Fully consolidated\nADIB (UK) Limited United Kingdom 100 Other Services Fully consolidated Fully consolidated\nADIB Holdings (Jersey) Ltd* (under liquidation)British Channel \nIslands  - Special Purpose Vehicle Full

"Burooj Properties LLC is a real estate subsidiary of Abu Dhabi Islamic Bank (ADIB). It is mentioned in the ADIB annual reports. Here is some information about Burooj Properties LLC:\n\n1. Provision for Impairment: The annual report states that the provision for impairment includes nil (2020: AED 31,773 thousand) pertaining to Burooj Properties LLC.\n\n2. Real Estate Investments: Burooj Properties LLC is involved in real estate investments. The report mentions investment properties and development properties associated with Burooj Properties LLC.\n\n3. Historical Activities: In the 2018 annual report, it is mentioned that due to the conditions in the real estate market in the UAE, ADIB deemed it prudent to increase impairments relating to Burooj's historical activities by a further AED 61.8 million in 2018.\n\nPlease note that the information provided is based on the available annual reports and may not represent the complete scope of Burooj Properties LLC's activities. For more detail

In [21]:
result = agent.run('what were the investments in 2023 in csv ')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'text': 'investments'}`


[Document(page_content='properties. Investment properties are measured at cost less any accumulated depreciation and any accumulated impairment losses. \nDepreciation is charged on a straight-line basis over the assets’ estimated useful lives. The useful life of buildings is estimated to be 25 - 40 \nyears.\nInvestment properties are derecognized when either they have been disposed of or when the investment properties are permanently \nwithdrawn from use and no future economic benefits are expected from their disposal. Any gains or losses on the retirement or disposal of an \ninvestment property are recognized in the consolidated income statement.', metadata={'page': 99, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='The Group has entered into commercial property lease arrangements on its investment property portfolio. The Gro

[Document(page_content='properties. Investment properties are measured at cost less any accumulated depreciation and any accumulated impairment losses. \nDepreciation is charged on a straight-line basis over the assets’ estimated useful lives. The useful life of buildings is estimated to be 25 - 40 \nyears.\nInvestment properties are derecognized when either they have been disposed of or when the investment properties are permanently \nwithdrawn from use and no future economic benefits are expected from their disposal. Any gains or losses on the retirement or disposal of an \ninvestment property are recognized in the consolidated income statement.', metadata={'page': 99, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='The Group has entered into commercial property lease arrangements on its investment property portfolio. The Group has determined, based \non an evaluation of the terms and conditions of the arrangements, that it retains all

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4130 tokens (3991 in the messages, 139 in the functions). Please reduce the length of the messages or functions.